# 3.7 宽表聚合（每公司每季度一行）

目标：
- 将 `segments_features`（Prepared Remarks / Q&A 两行）转换为每个 `ticker + quarter` 一行（Wide format）
- 所有特征展开为 `*_pr`、`*_qa`（例如 `smog_index_pr`, `smog_index_qa`）
- 非特征字段尽量保留原信息（如 `timestamp/url/source_file/id` 也展开为 `*_pr`、`*_qa`）
- 结果写入**新的数据库**，不改原 `earnings_calls_features.db`

说明：
- 一行 = 一个公司 × 一个季度
- 非特征信息：`ticker`, `quarter`, `section`, `timestamp`, `url`, `source_file`, `id`
- 其中 `section/timestamp/url/source_file/id` 在宽表中仅保留**单列**（优先取 PR，否则取 QA），不作为特征
- 新数据库：`earnings_calls_features_wide.db`
- 新表名：`call_features_wide`

In [1]:
# ========== 配置 ==========

import sqlite3
from pathlib import Path

import numpy as np
import pandas as pd

PROJECT_ROOT = Path("..").resolve()
SOURCE_DB = PROJECT_ROOT / "data" / "earnings_calls_features.db"
WIDE_DB = PROJECT_ROOT / "data" / "earnings_calls_features_wide.db"
OUT_TABLE = "call_features_wide"
OUT_CSV = PROJECT_ROOT / "data" / "call_features_wide.csv"

print("SOURCE_DB:", SOURCE_DB)
print("WIDE_DB:", WIDE_DB)
print("OUT_TABLE:", OUT_TABLE)
print("OUT_CSV:", OUT_CSV)

SOURCE_DB: /Users/xinyuewang/Desktop/1.27/data/earnings_calls_features.db
WIDE_DB: /Users/xinyuewang/Desktop/1.27/data/earnings_calls_features_wide.db
OUT_TABLE: call_features_wide
OUT_CSV: /Users/xinyuewang/Desktop/1.27/data/call_features_wide.csv


In [2]:
# ========== 1) 读取原始 segment-level 特征 ==========

conn = sqlite3.connect(SOURCE_DB)
df = pd.read_sql_query("SELECT * FROM segments_features", conn)
conn.close()

print(f"segments_features 行数: {len(df)}")
print("section 分布:")
print(df["section"].value_counts(dropna=False).to_string())
df.head(3)

segments_features 行数: 2374
section 分布:
section
Prepared Remarks    1213
Q&A                 1161


,id,ticker,quarter,section,timestamp,url,source_file,automated_readability,coleman_liau,dale_chall,...,adverb_ratio_zscore,lm_net_sentiment_zscore,lm_uncertainty_zscore,lm_litigious_zscore,lm_subjectivity_zscore,lm_modal_strong_zscore,lm_modal_weak_zscore,eps_beat,next_quarter_estimated,next_quarter_actual
0,75,AAPL,2015-Q1,Prepared Remarks,2015-01-27 17:00,https://seekingalpha.com/article/2856006-apple...,transcripts/AAPL/Apple's (AAPL) CEO Tim Cook o...,11.676568,11.167627,10.767866,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.54,0.58
1,77,AAPL,2015-Q1,Prepared Remarks,2015-04-27 17:00,https://seekingalpha.com/article/2856006-apple...,transcripts/AAPL/Apple's (AAPL) CEO Tim Cook o...,10.658642,10.213201,10.384815,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.54,0.58
2,76,AAPL,2015-Q1,Q&A,2015-01-27 17:00,https://seekingalpha.com/article/2856006-apple...,transcripts/AAPL/Apple's (AAPL) CEO Tim Cook o...,8.272458,7.819903,8.695382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.54,0.58


In [3]:
# ========== 2) 预处理：标准化 section，并定义“特征列/非特征列” ==========

def normalize_section(s: str) -> str:
    t = str(s).strip().lower()
    if "prepared" in t:
        return "pr"
    if "q&a" in t or t == "qa" or "q and a" in t:
        return "qa"
    return "other"

df["section_key"] = df["section"].apply(normalize_section)
df = df[df["section_key"].isin(["pr", "qa"])].copy()

# 保留原始 section 文本，作为非特征信息输出为 section_pr/section_qa
df["section_raw"] = df["section"]

# 这些字段按 quarter 保留为“非特征信息”
key_cols = ["ticker", "quarter", "section_key"]
label_cols = [c for c in ["eps_beat", "next_quarter_estimated", "next_quarter_actual"] if c in df.columns]
meta_section_cols = [c for c in ["section_raw", "id", "timestamp", "url", "source_file"] if c in df.columns]

# 其余列都作为 feature，后续展开为 _pr/_qa
exclude = set(["section"] + key_cols + label_cols + meta_section_cols)
feature_cols = [c for c in df.columns if c not in exclude]

# overall 是文本等级列，不参与数值聚合
feature_cols = [c for c in feature_cols if c != "overall"]

print(f"section 后保留行数: {len(df)}")
print(f"feature 列数: {len(feature_cols)}")
print(f"meta_section 列: {meta_section_cols}")
print(f"label 列: {label_cols}")

section 后保留行数: 2374
feature 列数: 32
meta_section 列: ['section_raw', 'id', 'timestamp', 'url', 'source_file']
label 列: ['eps_beat', 'next_quarter_estimated', 'next_quarter_actual']


In [4]:
# ========== 3) 转宽表：一行一季度，特征/元信息全部展开为 _pr/_qa ==========

index_cols = ["ticker", "quarter"]

# 先对每个 ticker-quarter-section 去重（防止个别重复）
agg_map = {c: "mean" for c in feature_cols if c in df.columns}
for c in meta_section_cols:
    agg_map[c] = "first"
for c in label_cols:
    agg_map[c] = "first"

df_grp = (
    df.groupby(index_cols + ["section_key"], as_index=False)
      .agg(agg_map)
)

# 特征列 pivot
wide_feat = df_grp.pivot_table(
    index=index_cols,
    columns="section_key",
    values=feature_cols,
    aggfunc="mean"
)
wide_feat.columns = [f"{feat}_{sec}" for feat, sec in wide_feat.columns]

# 非特征元信息也按 section 展开（section/timestamp/url/source_file/id）
if meta_section_cols:
    wide_meta = df_grp.pivot_table(
        index=index_cols,
        columns="section_key",
        values=meta_section_cols,
        aggfunc="first"
    )
    wide_meta.columns = [f"{m}_{sec}" for m, sec in wide_meta.columns]
else:
    wide_meta = pd.DataFrame(index=wide_feat.index)

# 标签按 ticker+quarter 保留一份
if label_cols:
    wide_label = (
        df_grp[index_cols + label_cols]
        .drop_duplicates(subset=index_cols)
        .set_index(index_cols)
    )
else:
    wide_label = pd.DataFrame(index=wide_feat.index)

wide = pd.concat([wide_feat, wide_meta, wide_label], axis=1).reset_index()

# ---- 非特征元信息合并成单列（优先 PR，否则 QA）----
for m in ["id", "timestamp", "url", "source_file"]:
    pr_col, qa_col = f"{m}_pr", f"{m}_qa"
    if pr_col in wide.columns or qa_col in wide.columns:
        left = wide[pr_col] if pr_col in wide.columns else pd.Series([None] * len(wide))
        right = wide[qa_col] if qa_col in wide.columns else pd.Series([None] * len(wide))
        wide[m] = left.combine_first(right)

# section 也合并成单列
sec_pr_col, sec_qa_col = "section_raw_pr", "section_raw_qa"
if sec_pr_col in wide.columns or sec_qa_col in wide.columns:
    sec_pr = wide[sec_pr_col] if sec_pr_col in wide.columns else pd.Series([None] * len(wide))
    sec_qa = wide[sec_qa_col] if sec_qa_col in wide.columns else pd.Series([None] * len(wide))
    wide["section"] = np.where(sec_pr.notna() & sec_qa.notna(), "PR+QA",
                          np.where(sec_pr.notna(), "Prepared Remarks",
                          np.where(sec_qa.notna(), "Q&A", None)))

# 删除分段版元信息列
drop_meta_split = [
    c for c in [
        "id_pr", "id_qa",
        "timestamp_pr", "timestamp_qa",
        "url_pr", "url_qa",
        "source_file_pr", "source_file_qa",
        "section_raw_pr", "section_raw_qa"
    ] if c in wide.columns
]
wide = wide.drop(columns=drop_meta_split)

print(f"宽表行数: {len(wide)}")
print(f"宽表列数: {len(wide.columns)}")
wide.head(3)

宽表行数: 1201
宽表列数: 74


,ticker,quarter,adverb_ratio_pr,adverb_ratio_qa,adverb_ratio_zscore_pr,adverb_ratio_zscore_qa,automated_readability_pr,automated_readability_qa,coleman_liau_pr,coleman_liau_qa,...,words_per_sentence_zscore_pr,words_per_sentence_zscore_qa,eps_beat,next_quarter_estimated,next_quarter_actual,id,timestamp,url,source_file,section
0,AAPL,2015-Q1,0.010684,0.016920,NaN,NaN,11.167605,8.264663,10.690414,7.638392,...,NaN,NaN,1.0,0.54,0.58,75.0,2015-01-27 17:00,https://seekingalpha.com/article/2856006-apple...,transcripts/AAPL/Apple's (AAPL) CEO Tim Cook o...,PR+QA
1,AAPL,2015-Q3,0.010441,0.016761,NaN,NaN,11.087886,8.183379,10.666760,7.877953,...,NaN,NaN,1.0,0.47,0.49,21.0,2015-07-21 17:00,https://seekingalpha.com/article/3344115-apple...,transcripts/AAPL/Apple (AAPL) Timothy D. Cook ...,PR+QA
2,AAPL,2015-Q4,0.013018,0.019011,NaN,NaN,11.441274,8.946565,10.637844,7.904456,...,NaN,NaN,1.0,0.81,0.82,79.0,2015-10-27 17:00,https://seekingalpha.com/article/3611256-apple...,transcripts/AAPL/Apple's (AAPL) CEO Tim Cook o...,PR+QA


In [5]:
# ========== 4) 列检查：确认是 _pr/_qa 宽表 ==========

sample_cols = [
    c for c in [
        "smog_index_pr", "smog_index_qa",
        "lm_positive_pr", "lm_positive_qa",
        "section", "timestamp", "url", "source_file", "id",
        "eps_beat"
    ] if c in wide.columns
]

print("示例列:", sample_cols)
print("\n前 3 行:")
wide.head(3)

示例列: ['smog_index_pr', 'smog_index_qa', 'lm_positive_pr', 'lm_positive_qa', 'section', 'timestamp', 'url', 'source_file', 'id', 'eps_beat']

前 3 行:


,ticker,quarter,adverb_ratio_pr,adverb_ratio_qa,adverb_ratio_zscore_pr,adverb_ratio_zscore_qa,automated_readability_pr,automated_readability_qa,coleman_liau_pr,coleman_liau_qa,...,words_per_sentence_zscore_pr,words_per_sentence_zscore_qa,eps_beat,next_quarter_estimated,next_quarter_actual,id,timestamp,url,source_file,section
0,AAPL,2015-Q1,0.010684,0.016920,NaN,NaN,11.167605,8.264663,10.690414,7.638392,...,NaN,NaN,1.0,0.54,0.58,75.0,2015-01-27 17:00,https://seekingalpha.com/article/2856006-apple...,transcripts/AAPL/Apple's (AAPL) CEO Tim Cook o...,PR+QA
1,AAPL,2015-Q3,0.010441,0.016761,NaN,NaN,11.087886,8.183379,10.666760,7.877953,...,NaN,NaN,1.0,0.47,0.49,21.0,2015-07-21 17:00,https://seekingalpha.com/article/3344115-apple...,transcripts/AAPL/Apple (AAPL) Timothy D. Cook ...,PR+QA
2,AAPL,2015-Q4,0.013018,0.019011,NaN,NaN,11.441274,8.946565,10.637844,7.904456,...,NaN,NaN,1.0,0.81,0.82,79.0,2015-10-27 17:00,https://seekingalpha.com/article/3611256-apple...,transcripts/AAPL/Apple's (AAPL) CEO Tim Cook o...,PR+QA


In [6]:
# ========== 5) 质量检查：是否 PR/QA 齐全 ==========

needed_pairs = [
    ("smog_index_pr", "smog_index_qa"),
    ("lm_net_sentiment_pr", "lm_net_sentiment_qa"),
]

for pr_col, qa_col in needed_pairs:
    if pr_col in wide.columns and qa_col in wide.columns:
        miss_pr = wide[pr_col].isna().sum()
        miss_qa = wide[qa_col].isna().sum()
        print(f"{pr_col} 缺失: {miss_pr}, {qa_col} 缺失: {miss_qa}")

print("\n每公司季度行数（应接近季度数）:")
print(wide.groupby("ticker")["quarter"].nunique().sort_values(ascending=False).to_string())

smog_index_pr 缺失: 0, smog_index_qa 缺失: 52
lm_net_sentiment_pr 缺失: 0, lm_net_sentiment_qa 缺失: 52

每公司季度行数（应接近季度数）:
ticker
JPM     44
DAL     44
TSLA    44
MTB     44
ADBE    44
INTC    44
GS      44
WFC     44
BAC     44
AMD     44
AMZN    44
AAPL    43
PNC     43
TSM     43
BLK     43
BK      43
MS      43
STT     43
C       43
QCOM    43
BR      42
NVDA    42
NOW     42
MSFT    42
NFLX    42
AVGO    41
SHOP    40
CRM     39


In [7]:
# ========== 6) 写入“新数据库” + 导出 CSV ==========

conn = sqlite3.connect(WIDE_DB)
wide.to_sql(OUT_TABLE, conn, if_exists="replace", index=False)
conn.close()

wide.to_csv(OUT_CSV, index=False)

print("已写入新数据库:", WIDE_DB)
print("表名:", OUT_TABLE)
print("已导出 CSV:", OUT_CSV)
print(f"最终: {len(wide)} 行, {len(wide.columns)} 列")

已写入新数据库: /Users/xinyuewang/Desktop/1.27/data/earnings_calls_features_wide.db
表名: call_features_wide
已导出 CSV: /Users/xinyuewang/Desktop/1.27/data/call_features_wide.csv
最终: 1201 行, 74 列
